In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.impute import SimpleImputer

from predicting_odor_from_molecular_structure import *


df = goodscents()
df

,TGSC ID,CID,Concentration %,Solvent,MolecularWeight,IsomericSMILES,IUPACName,name,Labels
0,1000111,7476,100.0,NaN,150.17,CC(=O)C1=CC=C(C=C1)OC,1-(4-methoxyphenyl)ethanone,4'-methoxyacetophenone,"[sweet, vanilla, cherry maraschino cherry, pow..."
1,1031871,7478,10.0,dipropylene glycol,152.15,COC1=CC=C(C=C1)C(=O)O,4-methoxybenzoic acid,4-methoxybenzoic acid,"[phenolic, animal, fecal, medicinal]"
2,1009281,7501,0.1,triacetin,104.15,C=CC1=CC=CC=C1,styrene,styrene,"[sweet, plastic, floral, balsamic]"
3,1001651,244,100.0,NaN,108.14,C1=CC=C(C=C1)CO,phenylmethanol,benzyl alcohol,"[sweet, floral, rose, fruity, phenolic, balsam..."
4,1001491,240,10.0,dipropylene glycol,106.12,C1=CC=C(C=C1)C=O,benzaldehyde,benzaldehyde,"[sweet, cherry, cherry maraschino cherry, nutt..."
...,...,...,...,...,...,...,...,...,...
4621,1020461,11564539,NaN,NaN,170.29,CCCCCCCC(=C)C(C)O,3-methylidenedecan-2-ol,3-methylidenedecan-2-ol,"[flowers, orange]"
4622,1027261,46195361,1.0,dipropylene glycol,146.25,CCC1OCCC(S1)C,"2-ethyl-4-methyl-1,3-oxathiane",schembl2678282,"[vegetable, floral, fruity, apricot, green, tr..."
4623,1028151,46195364,0.1,propylene glycol,174.31,CCCCC1OCCC(S1)C,"2-butyl-4-methyl-1,3-oxathiane",schembl2676606,"[spicy, leafy, peppery, banana, fruity, oily, ..."
4624,1554361,88571201,NaN,NaN,114.19,CC[C@@H](C)CCC=O,(4R)-4-methylhexanal,(r)-4-methylhexanal,"[flowery, fresh, green, warm]"


In [2]:
df = reduce_df(df, minimum=30)
df.shape

(4506, 9)

In [3]:
from sklearn.preprocessing import MultiLabelBinarizer

classes = label_frequencies(df, labels_column='Labels').Label.to_list()
mlb = MultiLabelBinarizer(classes=classes)
Y = mlb.fit_transform(df['Labels'])

mlb, Y.shape

(MultiLabelBinarizer(classes=['fruity', 'green', 'sweet', 'floral', 'woody',
                              'herbal', 'fatty', 'fresh', 'waxy', 'spicy',
                              'citrus', 'rose', 'sulfurous', 'earthy', 'nutty',
                              'tropical', 'odorless', 'balsamic', 'oily',
                              'minty', 'vegetable', 'powdery', 'musty', 'meaty',
                              'creamy', 'pineapple', 'apple', 'aldehydic',
                              'roasted', 'phenolic', ...]),
 (4506, 133))

In [5]:
dfx = create_all_features(df)
dfx

,TGSC ID,CID,Concentration %,Solvent,MolecularWeight,IsomericSMILES,IUPACName,name,Labels,mol,morgan_fp,daylight_fp
0,1000111,7476,100.0,NaN,150.17,CC(=O)C1=CC=C(C=C1)OC,1-(4-methoxyphenyl)ethanone,4'-methoxyacetophenone,"[sweet, vanilla, powdery, anisic, balsamic, ha...",<rdkit.Chem.rdchem.Mol object at 0x0000017D3AC...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1031871,7478,10.0,dipropylene glycol,152.15,COC1=CC=C(C=C1)C(=O)O,4-methoxybenzoic acid,4-methoxybenzoic acid,"[phenolic, animal, medicinal]",<rdkit.Chem.rdchem.Mol object at 0x0000017D3AC...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1009281,7501,0.1,triacetin,104.15,C=CC1=CC=CC=C1,styrene,styrene,"[sweet, floral, balsamic]",<rdkit.Chem.rdchem.Mol object at 0x0000017D3AC...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1001651,244,100.0,NaN,108.14,C1=CC=C(C=C1)CO,phenylmethanol,benzyl alcohol,"[sweet, floral, rose, fruity, phenolic, balsam...",<rdkit.Chem.rdchem.Mol object at 0x0000017D3AC...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1001491,240,10.0,dipropylene glycol,106.12,C1=CC=C(C=C1)C=O,benzaldehyde,benzaldehyde,"[sweet, cherry, nutty, fruity, powdery, almond...",<rdkit.Chem.rdchem.Mol object at 0x0000017D3AC...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
4621,1020461,11564539,NaN,NaN,170.29,CCCCCCCC(=C)C(C)O,3-methylidenedecan-2-ol,3-methylidenedecan-2-ol,[orange],<rdkit.Chem.rdchem.Mol object at 0x0000017D3BD...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4622,1027261,46195361,1.0,dipropylene glycol,146.25,CCC1OCCC(S1)C,"2-ethyl-4-methyl-1,3-oxathiane",schembl2678282,"[vegetable, floral, fruity, apricot, green, tr...",<rdkit.Chem.rdchem.Mol object at 0x0000017D3BD...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4623,1028151,46195364,0.1,propylene glycol,174.31,CCCCC1OCCC(S1)C,"2-butyl-4-methyl-1,3-oxathiane",schembl2676606,"[spicy, leafy, peppery, banana, fruity, oily, ...",<rdkit.Chem.rdchem.Mol object at 0x0000017D3BD...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ..."
4624,1554361,88571201,NaN,NaN,114.19,CC[C@@H](C)CCC=O,(4R)-4-methylhexanal,(r)-4-methylhexanal,"[fresh, green, warm]",<rdkit.Chem.rdchem.Mol object at 0x0000017D3BD...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [6]:
mordred = create_mordred_features(dfx)
mordred

100%|██████████| 4506/4506 [00:49<00:00, 90.87it/s] 


,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,7.956514,7.451864,0,0,13.619695,2.250875,4.501749,13.619695,1.238154,3.286693,...,8.906935,40.567492,150.068080,7.146099,162,13,50.0,55.0,4.583333,2.611111
1,7.956514,7.451864,1,0,13.619695,2.250875,4.501749,13.619695,1.238154,3.286693,...,8.906935,40.567492,152.047344,8.002492,162,13,50.0,55.0,4.583333,2.611111
2,5.656854,5.427660,0,0,10.424292,2.135779,4.271558,10.424292,1.303037,2.969338,...,8.298291,35.247635,104.062600,6.503913,64,7,34.0,36.0,2.611111,2.000000
3,5.656854,5.427660,0,0,10.424292,2.135779,4.271558,10.424292,1.303037,2.969338,...,8.298291,35.247635,108.057515,6.753595,64,7,34.0,36.0,2.611111,2.000000
4,5.656854,5.427660,0,0,10.424292,2.135779,4.271558,10.424292,1.303037,2.969338,...,8.298291,35.247635,106.041865,7.574419,64,7,34.0,36.0,2.611111,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4621,8.065904,7.893227,0,0,13.812926,2.092177,4.184353,13.812926,1.151077,3.306818,...,8.302266,40.156271,170.167065,5.004914,256,11,46.0,46.0,5.722222,3.027778
4622,6.473351,6.191587,0,0,11.142106,2.200122,4.400244,11.142106,1.238012,3.089776,...,8.606851,37.313444,146.076536,6.351154,88,9,40.0,43.0,3.472222,2.166667
4623,7.887564,7.276324,0,0,13.649148,2.212234,4.424468,13.649148,1.240832,3.278334,...,8.711608,40.046626,174.107836,6.003718,170,11,48.0,51.0,3.972222,2.666667
4624,5.059137,5.436502,0,0,9.40926,1.989044,3.978088,9.40926,1.176158,2.88852,...,7.623153,33.372552,114.104465,5.186567,76,6,28.0,27.0,4.111111,2.166667


In [7]:
from sklearn.impute import KNNImputer

mordred_features_clean = mordred.dropna(axis=1, thresh=int(0.8*len(dfx)))
imputer = KNNImputer()
mordred_imputed = imputer.fit_transform(mordred_features_clean)
mordred_imputed

array([[ 7.95651408,  7.45186446,  0.        , ..., 55.        ,
         4.58333333,  2.61111111],
       [ 7.95651408,  7.45186446,  1.        , ..., 55.        ,
         4.58333333,  2.61111111],
       [ 5.65685425,  5.42766007,  0.        , ..., 36.        ,
         2.61111111,  2.        ],
       ...,
       [ 7.88756439,  7.27632423,  0.        , ..., 51.        ,
         3.97222222,  2.66666667],
       [ 5.05913727,  5.43650183,  0.        , ..., 27.        ,
         4.11111111,  2.16666667],
       [ 5.05913727,  5.43650183,  0.        , ..., 27.        ,
         4.11111111,  2.16666667]])

In [8]:
# Morgan i Daylight już są jako listy 0/1 w df
X_morgan = np.array(df['morgan_fp'].tolist())
X_daylight = np.array(df['daylight_fp'].tolist())

# # Mordred - imputacja (SimpleImputer na medianie)
# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(strategy='median')
# mordred_imputed = imputer.fit_transform(mordred_features)

feature_sets = {
    'Morgan': X_morgan,
    'Daylight': X_daylight,
    'Mordred': mordred_imputed,
}


feature_sets


{'Morgan': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0]]),
 'Daylight': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 'Mordred': array([[ 7.95651408,  7.45186446,  0.        , ..., 55.        ,
          4.58333333,  2.61111111],
        [ 7.95651408,  7.45186446,  1.        , ..., 55.        ,
          4.58333333,  2.61111111],
        [ 5.65685425,  5.42766007,  0.        , ..., 36.        ,
          2.61111111,  2.        ],
        ...,
        [ 7.88756439,  7.27632423,  0.        , ..., 51.        ,
          3.97222222,  2.66666667],
        [ 5.05913727,  5.43650183,  0.        , ..., 27.        ,
          4.11111111,  2.16666667],
        [ 5.059137

In [9]:
indices = np.arange(len(df))
# Dodaj indeks jako ostatnią kolumnę do cech (np. Mordred)
X_with_idx = np.concatenate([mordred_imputed, indices.reshape(-1, 1)], axis=1)
X_train, y_train, X_test, y_test = iterative_train_test_split(X_with_idx, Y, test_size=0.2)
train_indices = X_train[:, -1].astype(int)
test_indices = X_test[:, -1].astype(int)

def get_split(X, train_idx, test_idx):
    return X[train_idx], X[test_idx]


In [10]:
model_types = {
    'RF': lambda: RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'Chain': lambda: ClassifierChain(RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))
}

results = []
models = {}

for feat_name, X in feature_sets.items():
    X_train_, X_test_ = get_split(X, train_indices, test_indices)
    for model_name, model_init in model_types.items():
        key = f"{feat_name}_{model_name}"
        print(f"Trenuję {key} ...")
        model = model_init()
        model.fit(X_train_, y_train)
        y_pred = model.predict(X_test_)
        if hasattr(y_pred, "toarray"):
            y_pred = y_pred.toarray()
        # Metryki per-label
        precision = precision_score(y_test, y_pred, average=None, zero_division=0)
        recall = recall_score(y_test, y_pred, average=None, zero_division=0)
        f1 = f1_score(y_test, y_pred, average=None, zero_division=0)
        results.append({
            'model': key,
            'precision': precision,
            'recall': recall,
            'f1': f1
        })
        # Zapisz wytrenowany model i predykcje do późniejszej inferencji
        models[key] = {
            'model': model,
            'y_pred': y_pred,
            'X_test': X_test_,
            'y_test': y_test,
            'test_indices': test_indices
        }


Trenuję Morgan_RF ...
Trenuję Morgan_Chain ...
Trenuję Daylight_RF ...
Trenuję Daylight_Chain ...
Trenuję Mordred_RF ...
Trenuję Mordred_Chain ...


In [11]:
model_key = 'Mordred_RF'  # dowolny z 6
m = models[model_key]
sample_indices = np.random.choice(len(m['test_indices']), size=5, replace=False)
for idx in sample_indices:
    original_idx = m['test_indices'][idx]
    molecule_name = df.iloc[original_idx]['name']
    true_labels = mlb.inverse_transform(m['y_test'][idx].reshape(1, -1))[0]
    pred_labels = mlb.inverse_transform(m['y_pred'][idx].reshape(1, -1))[0]
    print(f"Idx: {original_idx} | Nazwa: {molecule_name}")
    print(f"  Prawdziwe etykiety: {true_labels}")
    print(f"  Predykowane etykiety: {pred_labels}")
    print("-" * 40)


Idx: 3640 | Nazwa: menthadienyl acetate
  Prawdziwe etykiety: ('fruity', 'minty')
  Predykowane etykiety: ('fruity', 'green', 'sweet', 'herbal', 'minty')
----------------------------------------
Idx: 625 | Nazwa: 25037-13-0
  Prawdziwe etykiety: ('green', 'floral', 'woody', 'herbal', 'citrus', 'vegetable', 'lavender', 'bergamot')
  Predykowane etykiety: ('green', 'floral', 'woody', 'herbal', 'citrus', 'vegetable', 'lavender', 'bergamot')
----------------------------------------
Idx: 4485 | Nazwa: schembl11471610
  Prawdziwe etykiety: ('woody', 'balsamic')
  Predykowane etykiety: ('woody', 'herbal')
----------------------------------------
Idx: 2867 | Nazwa: einecs 261-579-5
  Prawdziwe etykiety: ('woody', 'amber', 'sandalwood', 'vetiver')
  Predykowane etykiety: ('woody', 'amber')
----------------------------------------
Idx: 2553 | Nazwa: (2r,2'r,5's)-lilac aldehyde
  Prawdziwe etykiety: ('sweet',)
  Predykowane etykiety: ('fresh',)
----------------------------------------


In [12]:
model_key = 'Morgan_Chain'
idx = [r['model'] for r in results].index(model_key)
df_per_label = pd.DataFrame({
    'label': mlb.classes_,
    'precision': results[idx]['precision'],
    'recall': results[idx]['recall'],
    'f1': results[idx]['f1'],
})
df_per_label.head()

,label,precision,recall,f1
0,fruity,0.665000,0.532000,0.591111
1,green,0.629032,0.364486,0.461538
2,sweet,0.451327,0.253731,0.324841
3,floral,0.519685,0.352941,0.420382
4,woody,0.554054,0.297101,0.386792
